In [31]:
import pandas as pd
import numpy as np

model_input_path = "../data/model_input.parquet"

df_model_input = pd.read_parquet(model_input_path)

In [32]:
# --- Característica 1: Día de la Semana ---
# El ciclo tiene 7 periodos (de 0 a 6)
df_model_input['day_of_week_sin'] = np.sin(2 * np.pi * df_model_input['day_of_week'] / 7)
df_model_input['day_of_week_cos'] = np.cos(2 * np.pi * df_model_input['day_of_week'] / 7)

# --- Característica 2: Hora del Día ---
# El ciclo tiene 24 periodos (de 0 a 23)
df_model_input['hour_sin'] = np.sin(2 * np.pi * df_model_input['hour'] / 24)
df_model_input['hour_cos'] = np.cos(2 * np.pi * df_model_input['hour'] / 24)

# --- Característica 3: Mes del Año ---
# El ciclo tiene 12 periodos (de 1 a 12)
df_model_input['month_sin'] = np.sin(2 * np.pi * df_model_input['month'] / 12)
df_model_input['month_cos'] = np.cos(2 * np.pi * df_model_input['month'] / 12)


# Veamos las nuevas columnas que hemos creado
print(df_model_input[['day_of_week', 'day_of_week_sin', 'day_of_week_cos']].head())

   day_of_week  day_of_week_sin  day_of_week_cos
0            6        -0.781831          0.62349
1            6        -0.781831          0.62349
2            6        -0.781831          0.62349
3            6        -0.781831          0.62349
4            6        -0.781831          0.62349


In [33]:
# --- Característica 4: Demanda en la Hora Anterior (Lag de 1 hora) ---

# Ordenamos los datos para asegurarnos de que el retraso se calcula correctamente
# para cada ruta de autobús por separado.
df_model_input = df_model_input.sort_values(by=['bus_route', 'hour_timestamp'])

# Creamos la nueva columna. .shift(1) desplaza los datos una fila hacia abajo.
# Lo agrupamos por 'bus_route' para que el lag no "salte" de una ruta a otra.
df_model_input['ridership_lag_1h'] = df_model_input.groupby('bus_route', observed=False)['ridership_total'].shift(1)

# Veamos el resultado. Fíjate en la primera fila de cada ruta.
print(df_model_input[['bus_route', 'hour_timestamp', 'ridership_total', 'ridership_lag_1h']].head())

  bus_route      hour_timestamp  ridership_total  ridership_lag_1h
0       M15 2023-01-01 00:00:00              323               NaN
1       M15 2023-01-01 01:00:00              300             323.0
2       M15 2023-01-01 02:00:00              144             300.0
3       M15 2023-01-01 03:00:00              101             144.0
4       M15 2023-01-01 04:00:00               38             101.0


In [34]:
# --- Manejo de Valores Faltantes ---

# Guardamos el número de filas antes de limpiar para ver el cambio
filas_antes = len(df_model_input)
print(f"Número de filas antes de limpiar los NaN: {filas_antes}")

# Eliminamos todas las filas que contengan cualquier valor NaN
df_model_input = df_model_input.dropna()

filas_despues = len(df_model_input)
print(f"Número de filas después de limpiar los NaN: {filas_despues}")
print(f"Se eliminaron {filas_antes - filas_despues} filas.")

Número de filas antes de limpiar los NaN: 19632
Número de filas después de limpiar los NaN: 19629
Se eliminaron 3 filas.


In [35]:
# --- Característica 5: Demanda del Día Anterior (Lag de 24 horas) ---

# Usamos .shift(24) para obtener el valor de ridership de exactamente 24 horas antes
# para la misma ruta de autobús.
df_model_input['ridership_lag_24h'] = df_model_input.groupby('bus_route', observed=False)['ridership_total'].shift(24)

# --- Manejo de Valores Faltantes ---

# Ahora, vamos a limpiar los NaNs generados por este nuevo lag
filas_antes = len(df_model_input)
print(f"Filas antes de limpiar el lag de 24h: {filas_antes}")

df_model_input = df_model_input.dropna()

filas_despues = len(df_model_input)
print(f"Filas después de limpiar el lag de 24h: {filas_despues}")
print(f"Se eliminaron {filas_antes - filas_despues} filas.")

# Veamos el resultado
print("\nDataFrame con lag de 1h y 24h:")
print(df_model_input[['bus_route', 'hour_timestamp', 'ridership_total', 'ridership_lag_1h', 'ridership_lag_24h']].head())

Filas antes de limpiar el lag de 24h: 19629
Filas después de limpiar el lag de 24h: 19557
Se eliminaron 72 filas.

DataFrame con lag de 1h y 24h:
   bus_route      hour_timestamp  ridership_total  ridership_lag_1h  \
25       M15 2023-01-02 01:00:00               45              92.0   
26       M15 2023-01-02 02:00:00                8              45.0   
27       M15 2023-01-02 03:00:00               12               8.0   
28       M15 2023-01-02 04:00:00               51              12.0   
29       M15 2023-01-02 05:00:00              142              51.0   

    ridership_lag_24h  
25              300.0  
26              144.0  
27              101.0  
28               38.0  
29               60.0  


In [36]:
# --- Característica 6: Demanda de la semana anterior ---


df_model_input['ridership_lag_1_week'] = df_model_input.groupby('bus_route', observed=False)['ridership_total'].shift(24*7)

# --- Manejo de Valores Faltantes ---

# Ahora, vamos a limpiar los NaNs generados por este nuevo lag
filas_antes = len(df_model_input)
print(f"Filas antes de limpiar: {filas_antes}")

df_model_input = df_model_input.dropna()

filas_despues = len(df_model_input)
print(f"Filas después de limpiar el lag: {filas_despues}")
print(f"Se eliminaron {filas_antes - filas_despues} filas.")

# Veamos el resultado
print(df_model_input[['bus_route', 'hour_timestamp', 'ridership_total', 'ridership_lag_24h', 'ridership_lag_1_week']].head())

Filas antes de limpiar: 19557
Filas después de limpiar el lag: 19053
Se eliminaron 504 filas.
    bus_route      hour_timestamp  ridership_total  ridership_lag_24h  \
193       M15 2023-01-09 01:00:00               26               71.0   
194       M15 2023-01-09 02:00:00               11               46.0   
195       M15 2023-01-09 03:00:00               23               33.0   
196       M15 2023-01-09 04:00:00               76               34.0   
197       M15 2023-01-09 05:00:00              154               86.0   

     ridership_lag_1_week  
193                  45.0  
194                   8.0  
195                  12.0  
196                  51.0  
197                 142.0  


In [38]:
# --- Característica 7: Media Móvil de los últimos 7 días ---

# Define el tamaño de la ventana para 7 días (en horas)
WINDOW_SIZE_7D = 24*7  # <--- COMPLETA ESTO

# Calculamos la media móvil
df_model_input['ridership_roll_mean_7d'] = df_model_input.groupby('bus_route', observed=False)['ridership_total'].rolling(window=WINDOW_SIZE_7D).mean().reset_index(level=0, drop=True)

# Como siempre, las primeras filas no tendrán suficientes datos para llenar la ventana, así que limpiamos los NaN
df_model_input = df_model_input.dropna()

print(df_model_input[['bus_route', 'hour_timestamp', 'ridership_total', 'ridership_roll_mean_7d']].head())



    bus_route      hour_timestamp  ridership_total  ridership_roll_mean_7d
167       M15 2023-01-22 23:00:00              181              438.958333
168       M15 2023-01-23 00:00:00               82              438.601190
169       M15 2023-01-23 01:00:00               24              438.404762
170       M15 2023-01-23 02:00:00               13              438.303571
171       M15 2023-01-23 03:00:00               24              438.339286


In [39]:
# --- Feature 8: Precipitation Category ---

# Define the bin edges for our categories
bin_edges = [-1, 0, 1, 5, float('inf')] # from -1 to 0, >0 to 1, >1 to 5, and >5
bin_labels = ['0_no_rain', '1_light_rain', '2_moderate_rain', '3_heavy_rain']

# Use pd.cut to create the new column
df_model_input['precipitation_category'] = pd.cut(df_model_input['precipitation'], bins=bin_edges, labels=bin_labels)

# Let's see the result
print(df_model_input[['precipitation', 'precipitation_category']].head())

     precipitation precipitation_category
167            0.1           1_light_rain
168            0.6           1_light_rain
169            1.7        2_moderate_rain
170            1.6        2_moderate_rain
171            1.3        2_moderate_rain


In [40]:
# --- Feature 9: Temperature and Wind Interaction ---

# Create the interaction feature
df_model_input['temp_wind_interaction'] = df_model_input['temperature_2m'] / (df_model_input['wind_speed_10m'] + 1)

# Let's see the result
print(df_model_input[['temperature_2m', 'wind_speed_10m', 'temp_wind_interaction']].head())

     temperature_2m  wind_speed_10m  temp_wind_interaction
167             4.0             3.4               0.909091
168             1.5             9.4               0.144231
169             1.8            11.9               0.139535
170             1.8            12.4               0.134328
171             1.9            13.4               0.131944


In [42]:
# --- Característica 10 y 11: Proximidad a Días Festivos ---

# Nos aseguramos de que el DataFrame esté ordenado por fecha
df_model_input = df_model_input.sort_values(by='hour_timestamp')

# Creamos una serie que solo contiene las fechas de los días festivos
holidays = df_model_input[df_model_input['is_holiday']].index

# Creamos una serie temporal vacía con el mismo índice que nuestro DataFrame
# para almacenar los resultados
ts = pd.Series(index=df_model_input.index, dtype='datetime64[ns]')

# Para cada día festivo, lo marcamos en nuestra nueva serie temporal
ts.loc[holidays] = df_model_input.loc[holidays, 'hour_timestamp']

# Usamos 'forward fill' y 'backward fill' para propagar las fechas de los festivos
# a todas las filas del DataFrame
# ffill() rellena hacia adelante (dándonos la fecha del último festivo)
# bfill() rellena hacia atrás (dándonos la fecha del próximo festivo)
last_holiday = ts.ffill()
next_holiday = ts.bfill()

# Calculamos la diferencia en días entre la fecha actual y el festivo más cercano
# y lo convertimos a un número entero de días
df_model_input['days_since_last_holiday'] = (df_model_input['hour_timestamp'] - last_holiday).dt.days
df_model_input['days_until_next_holiday'] = (next_holiday - df_model_input['hour_timestamp']).dt.days

# Rellenamos cualquier posible NaN al principio o al final del dataset con un valor alto
df_model_input['days_since_last_holiday'].fillna(999, inplace=True)
df_model_input['days_until_next_holiday'].fillna(999, inplace=True)


# Veamos el resultado
print(df_model_input[['hour_timestamp', 'is_holiday', 'days_since_last_holiday', 'days_until_next_holiday']].head())

          hour_timestamp  is_holiday  days_since_last_holiday  \
167  2023-01-22 23:00:00       False                    999.0   
8566 2023-01-22 23:00:00       False                    999.0   
8567 2023-01-23 00:00:00       False                    999.0   
168  2023-01-23 00:00:00       False                    999.0   
169  2023-01-23 01:00:00       False                    999.0   

      days_until_next_holiday  
167                      20.0  
8566                     20.0  
8567                     20.0  
168                      20.0  
169                      19.0  
